In [2]:
import requests
import psycopg2
from datetime import datetime
import time

# WeatherAPI.com API endpoint and API key
url = "http://api.weatherapi.com/v1/current.json"
api_key = "6d8ff939d2684f3aa6b213101232403"

# PostgreSQL database connection parameters
db_host = "localhost"
db_port = "5432"
db_name = "weather"
db_user = "postgres"
db_password = "secret"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    database=db_name,
    user=db_user,
    password=db_password
)

# Create a cursor object to execute PostgreSQL queries
cur = conn.cursor()

# Define the SQL query to create a weather table
weather_table_query = """
CREATE TABLE IF NOT EXISTS weather_tb (
    city VARCHAR(100) NOT NULL,
    country VARCHAR(50) NOT NULL,
    temperature FLOAT NOT NULL,
    humidity FLOAT NOT NULL,
    wind_speed FLOAT NOT NULL,
    pressure FLOAT NOT NULL,
    visibility FLOAT NOT NULL,
    timestamp TIMESTAMP NOT NULL DEFAULT NOW()
);
"""

# Execute the weather table query
cur.execute(weather_table_query)
conn.commit()

# Define the list of cities and countries to get weather data for
cities = [("New York", "US"), ("London", "UK"), ("Paris", "France")]

# Define the delay between each iteration of the loop (in seconds)
delay = 60

# Continuous loop to retrieve and insert weather data
while range(1,10):
    # Loop through the cities and get weather data from the WeatherAPI.com API
    for city, country in cities:
        # Define the API parameters
        params = {
            "key": api_key,
            "q": f"{city},{country}",
            "aqi": "no"
        }
        # Send the API request
        response = requests.get(url, params=params)
        # Parse the API response
        data = response.json()
        print(data)
        # Extract the weather data
        temperature = data["current"]["temp_c"]
        humidity = data["current"]["humidity"]
        wind_speed = data["current"]["wind_kph"]
        pressure = data["current"]["pressure_mb"]
        #cloud_cover = data["current"]["cloud"]
        visibility = data["current"]["vis_km"]
        # Insert the weather data into the weather table
        insert_query = """
        INSERT INTO weather_tb (city, country, temperature, humidity, wind_speed, pressure, visibility)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """
        cur.execute(insert_query, (city, country, temperature, humidity, wind_speed, pressure, visibility))
        conn.commit()
    # Wait for the delay before the next iteration of the loop
    time.sleep(delay)

# Close the database connection
conn.close()

{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.71, 'lon': -74.01, 'tz_id': 'America/New_York', 'localtime_epoch': 1680117329, 'localtime': '2023-03-29 15:15'}, 'current': {'temp_c': 11.1, 'condition': {}, 'wind_kph': 3.6, 'pressure_mb': 1021.0, 'humidity': 19, 'cloud': 0, 'feelslike_c': 9.6, 'vis_km': 16.0, 'uv': 4.0}}
{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.52, 'lon': -0.11, 'tz_id': 'Europe/London', 'localtime_epoch': 1680117329, 'localtime': '2023-03-29 20:15'}, 'current': {'temp_c': 12.0, 'condition': {}, 'wind_kph': 13.0, 'pressure_mb': 1004.0, 'humidity': 94, 'cloud': 75, 'feelslike_c': 9.7, 'vis_km': 4.0, 'uv': 1.0}}
{'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.87, 'lon': 2.33, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1680117329, 'localtime': '2023-03-29 21:15'}, 'current': {'temp_c': 19.0, 'condition'